In [25]:
import numpy as np
import matplotlib as plt
import pandas as pd
import os
import seaborn as sns

In [5]:
os. getcwd()
path = os. getcwd()

In [6]:
data1 = pd.read_csv(path + '/data/texas_counties1.csv')
data2 = pd.read_csv(path + '/data/texas_counties2.csv')
data3 = pd.read_csv(path + '/data/texas_counties3.csv')

In [7]:
frames = [data1,data2,data3]
master = pd.concat(frames)
master.describe()
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7290 entries, 0 to 2428
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       7290 non-null   int64  
 1   date             7290 non-null   object 
 2   sales            7288 non-null   object 
 3   dollar_vol       7288 non-null   object 
 4   avg_price        7288 non-null   object 
 5   med_price        7288 non-null   object 
 6   total_listings   5130 non-null   object 
 7   month_inventory  5130 non-null   float64
 8   county           7290 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 569.5+ KB


In [8]:
master.head()

,Unnamed: 0,date,sales,dollar_vol,avg_price,med_price,total_listings,month_inventory,county
0,0,Jan 2011,15,"1,122,600","74,840","50,000",344,31.0,Anderson
1,1,Feb 2011,20,"1,652,300","82,615","66,950",371,30.1,Anderson
2,2,Mar 2011,25,"3,013,550","120,542","92,000",372,27.1,Anderson
3,3,Apr 2011,25,"2,733,350","109,334","115,000",365,24.1,Anderson
4,4,May 2011,27,"2,239,161","82,932","70,000",387,23.8,Anderson


In [9]:
master['Unnamed: 0'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127])

In [10]:
master['date']

0       Jan 2011
1       Feb 2011
2       Mar 2011
3       Apr 2011
4       May 2011
          ...   
2424    Mar 2021
2425    Apr 2021
2426    May 2021
2427    Jun 2021
2428    Jul 2021
Name: date, Length: 7290, dtype: object

In [11]:
#remove 'Unnamed:0' column as it simply is another index for the month
master = master.drop(columns='Unnamed: 0')

#convert date column to datetime format & sort
master['date'] = pd.to_datetime(master['date'])

In [12]:
master.sort_values(by = 'date')

,date,sales,dollar_vol,avg_price,med_price,total_listings,month_inventory,county
0,2011-01-01,15,"1,122,600","74,840","50,000",344,31.0,Anderson
1664,2011-01-01,37,"4,405,050","119,055","107,500",NaN,NaN,Coryell
767,2011-01-01,"1,984","366,152,263","184,553","123,250",NaN,NaN,Harris
1536,2011-01-01,72,"16,445,947","228,416","203,500",NaN,NaN,Comal
894,2011-01-01,64,"10,603,874","165,686","146,000",NaN,NaN,Hays
...,...,...,...,...,...,...,...,...
127,2021-08-01,48,"10,438,222","217,463","172,500",89,2.4,Anderson
2431,2021-08-01,312,"115,686,454","370,790","345,140",362,1.3,Ellis
2301,2021-08-01,264,"99,587,752","377,226","305,000",732,3.0,Midland
1149,2021-08-01,388,"83,761,348","215,880","190,000",804,2.3,Hidalgo


In [17]:
#probabilistic PCA to infer NaN values
master[['total_listings','month_inventory']].isnull().values.any()

True

In [26]:
master.drop(['total_listings','month_inventory'], axis=1)
master.info

<bound method DataFrame.info of            date  sales   dollar_vol avg_price med_price total_listings  \
0    2011-01-01     15    1,122,600    74,840    50,000            344   
1    2011-02-01     20    1,652,300    82,615    66,950            371   
2    2011-03-01     25    3,013,550   120,542    92,000            372   
3    2011-04-01     25    2,733,350   109,334   115,000            365   
4    2011-05-01     27    2,239,161    82,932    70,000            387   
...         ...    ...          ...       ...       ...            ...   
2424 2021-03-01  1,447  542,948,174   375,223   300,000          1,462   
2425 2021-04-01  1,383  546,395,785   395,080   307,194          1,379   
2426 2021-05-01  1,505  589,443,326   391,657   311,340          1,363   
2427 2021-06-01  1,547  641,243,406   414,508   320,000          1,577   
2428 2021-07-01  1,489  621,533,727   417,417   333,500          2,030   

      month_inventory      county  
0                31.0    Anderson  
1      

In [27]:
master.info

<bound method DataFrame.info of            date  sales   dollar_vol avg_price med_price total_listings  \
0    2011-01-01     15    1,122,600    74,840    50,000            344   
1    2011-02-01     20    1,652,300    82,615    66,950            371   
2    2011-03-01     25    3,013,550   120,542    92,000            372   
3    2011-04-01     25    2,733,350   109,334   115,000            365   
4    2011-05-01     27    2,239,161    82,932    70,000            387   
...         ...    ...          ...       ...       ...            ...   
2424 2021-03-01  1,447  542,948,174   375,223   300,000          1,462   
2425 2021-04-01  1,383  546,395,785   395,080   307,194          1,379   
2426 2021-05-01  1,505  589,443,326   391,657   311,340          1,363   
2427 2021-06-01  1,547  641,243,406   414,508   320,000          1,577   
2428 2021-07-01  1,489  621,533,727   417,417   333,500          2,030   

      month_inventory      county  
0                31.0    Anderson  
1      

In [ ]:
#